# Importing ORACC Data from corpus.json
by Niek Veldhuis
UC Berkeley

March 2017

# TODO
* check that COFs are treated properly
* check that lines that continue into the next line (as in bilinguals) are captured completely. Such lines are indicated in the json by the the addition of 'l' (lower case L) to the reference (.ref).


# Introduction

Purpose of the code is to download [ORACC](http://oracc.org) JSON files that contain textual data and produce a `.csv` file with the relevant data for use in phylogenetics project. The JSON files contain all the transliteration and lemmatization data of an ORACC project (metadata are made available in a separate `.json` file). For an introduction to the various ORACC JSON files see the [ORACC Open Data](http://oracc.org/doc/opendata) page.

The resulting data file includes various elements of the ORACC data structure. The current code will output a file with the following fields: 

* id_line
* label
* lemma
* base
* extent
* scope

The fields `extent` and `scope` capture the number of missing lines or columns.

The selection of fields may be adjusted with standard `Pandas` functions.

## Notes
The current version of the script works with the `ijson` library. Documentation for [ijson](https://www.dataquest.io/blog/python-json-tutorial/), unfortunately, is extremely brief. Downloading the `.json` files from the ORACC server will take some time. The rest of the script is reasonably fast.

This notebook is written for **Python 3.5** with **Pandas 0.19** and **ijson 2.3**.

The notebook was written for the [Phylogenetics](https://github.com/ErinBecker/digital-humanities-phylogenetics) project with Erin Becker of [Data Carpentry](http://www.datacarpentry.org). The particular data selection and data manipulation performed in this notebook are inspired by the needs of that project (for instance, non-Sumerian words are filtered out). It should be fairly easy to adapt the notebook to the purposes of any other project that wishes to use [ORACC](http://oracc.org) data.

## Licensing
This notebook may be downloaded, used and adapted without any restrictions.

In [1]:
import pandas as pd   
import ijson
import urllib.request
import re
import tqdm

# Input List of Text IDs
Identify a list of text IDs (P, Q, and X numbers) in the directory `input`. The IDs are six-digit P, Q, or X numbers preceded by a project abbreviation in the format 'PROJECT/P######' or 'PROJECT/SUBPROJECT/Q######'. For example:
* dcclt/P117395
* etcsri/Q001203
* rinap/rinap1/Q003421

The list should be created with a flat text editor such as Textedit or Emacs, and the filename should end in `.txt`.

In [45]:
filename = input('Filename: ')

Filename: Q42_par.txt


In [46]:
textids = '../data/text_ids/' + filename
with open(textids, 'r') as f:
    pqxnos = f.readlines()
pqxnos = [x.strip() for x in pqxnos]
prefix = [x[:-7] + 'corpusjson/' + x[-7:] for x in pqxnos]
pqxnos = [x[-7:] for x in pqxnos]
pqxnos[:5], prefix[:5]


(['P346860', 'P272534', 'P253891', 'P235651', 'P225021'],
 ['dcclt/corpusjson/P346860',
  'dcclt/corpusjson/P272534',
  'dcclt/corpusjson/P253891',
  'dcclt/corpusjson/P235651',
  'dcclt/corpusjson/P225021'])

# Parse
The function `oraccjasonparser()` takes one argument (the **url** of the `.json` file). It looks for the prefix `textid` to retrieve the six-digit P, Q, or X number of the text artifact. Parsing the file sequentially the code looks for the places where a line starts (`'.type' = 'line-start'`) and where a word starts (`'.node' = 'l'`, where `l` is for "lemma"). At each level the code will retrieve the relevant data and create a list where each entry is a dictionary that represents a single word. 

Words not only include lemmatized words, but also unlemmatized and unlemmatizable words (such as breaks).

The dictionary includes the keys `id_line` and `id_word` that allow the user to reassemble words and lines in order.

In [47]:
def oraccjasonparser(url):
    d = urllib.request.urlopen(url)
    parser = ijson.parse(d)
    word_l = []
    word_d = {}
    line_start = False
    word_start = False
    nonx = False
    for prefix, event, value in parser:
        if prefix == 'textid':
            id_text = value
#            print("parsing " + value)
        if prefix.endswith('.type'):
            if value == 'line-start':
                line_start = True
            else:
                line_start = False
        if line_start:
            if prefix.endswith('.ref') and not word_start:
                id_line = value # id_line is a reference number for a line
                                # that includes the id_text (e.g. P123456.49)
            if prefix.endswith('.label'):
                label = value   # label is a human-readable line number of the format
                                # o ii 24' (obverse column 2 line 24')
        if prefix.endswith('node'):
            if value == 'l':
                word_start = True
                if not word_d == {}:
                    word_l.append(word_d) # append the previous word to the list
                word_d = {}               # and start a new dictionary
                word_d['id_text'] = id_text # provide each word with appropriate 
                word_d['id_line'] = id_line # text and line-ID
                word_d['label'] = label     # and the line label.
            else:
                word_start = False
        if word_start:
            if prefix.endswith('.ref'):
                word_d['id_word'] = value
            if prefix.endswith('.sig'):
                word_d['signature'] = value
            if '.f.' in prefix:
                category = re.sub('.*\.', '', prefix) # get element after the last dot of the prefix
                word_d[category] = value # copy each element into the dictionary
        if prefix.endswith('.type'):
            if value == 'nonx':
                nonx = True
            else:
                nonx = False
        if nonx:                         # this captures so-called $-lines with information
            if prefix.endswith('.ref'):  # about number of broken lines/columns.
                id_line = value          # $-lines have their own id_line.
            if prefix.endswith('.strict'):
                if value == '1':           # select only 'strict' $ lines
                    if not word_d == {}:
                        word_l.append(word_d)
                    word_d = {}
                    word_d['id_line'] = id_line
                    word_d['id_text'] = id_text
                else:
                    nonx = False
            if prefix.endswith('.extent'): # capture the three elements of strict $ lines
                word_d['extent'] = value   # namely extent, scope, and state.
            if prefix.endswith('.scope'):
                word_d['scope'] = value
            if prefix.endswith('.state'):
                word_d['state'] = value

    word_l.append(word_d)  # make sure that the last word is captured, too.
    return(word_l) # return a list of dictionaries, where each entry (dictionary) in
                   # the list represents a word.

# Call the Parser Function for Each Textid

In [48]:
url_prefix = "http://oracc.museum.upenn.edu/"
word_l = []
for id_text in tqdm.tqdm(prefix):   # tqdm creates a progress bar
    try:
        url = url_prefix + id_text + '.json'
        word_l.extend(oraccjasonparser(url))
    except:
        print(url + ' not available') 

 75%|███████▌  | 6/8 [00:00<00:00,  6.89it/s]

http://oracc.museum.upenn.edu/dcclt/corpusjson/P225021.json not available
http://oracc.museum.upenn.edu/dcclt/corpusjson/P250366.json not available


100%|██████████| 8/8 [00:01<00:00,  6.92it/s]

http://oracc.museum.upenn.edu/dcclt/corpusjson/P251685.json not available


# Transform the Data into a DataFrame

In [49]:
words = pd.DataFrame(word_l)
words.head()

,base,cf,cont,epos,form,gw,id_line,id_text,id_word,label,lang,morph,norm0,pos,sense,signature
0,urim₂{ki},Urim,NaN,SN,urim₂{ki},1,P346860.2,P346860,P346860.2.1,o 1,sux,~,Urim,SN,1,@dcclt%sux:urim₂{ki}=Urim[1//1]SN'SN$Urim/urim...
1,nibru{ki},Nibru,NaN,SN,nibru{ki},1,P346860.3,P346860,P346860.3.1,o 2,sux,~,Nibru,SN,1,@dcclt%sux:nibru{ki}=Nibru[1//1]SN'SN$Nibru/ni...
2,i₃-si-in{ki},Isin,NaN,SN,i₃-si-in{ki},1,P346860.4,P346860,P346860.4.1,o 3,sux,~,Isin,SN,1,@dcclt%sux:i₃-si-in{ki}=Isin[1//1]SN'SN$Isin/i...
3,NaN,NaN,NaN,NaN,x{ki},NaN,P346860.5,P346860,P346860.5.1,o 4,sux,NaN,NaN,u,NaN,NaN
4,a-šag₄,ašag,NaN,N,a-šag₄,field,P272534.3,P272534,P272534.3.1,o 1,sux,~,ašag,N,field,@dcclt%sux:a-šag₄=ašag[field//field]N'N$ašag/a...


# Remove Spaces and Commas from Guide Word and Sence
Spaces in Guide Word and Sense may cause trouble in computational methods in tokenization, or when saved in Comma Separated Values format. All spaces and commas are replaced by hyphens or nothing, respectively.

In [50]:
words = words.fillna('') # first replace Missing Values by empty string
words['sense'] = [x.replace(' ', '-') for x in words['sense']]
words['sense'] = [x.replace(',', '') for x in words['sense']]
words['gw'] = [x.replace(' ', '-') for x in words['gw']]
words['gw'] = [x.replace(',', '') for x in words['gw']]

The columns in the resulting DataFrame correspond to the elements of a full [ORACC](http://oracc.org) signature, plus information about text, line, and word ids:
* base (Sumerian only)
* cf (Citation Form)
* cont (continuation of the base; Sumerian only)
* epos (Effective Part of Speech)
* form (transliteration, omitting all flags such as indication of breakage)
* gw (Guide Word: main or first translation in standard dictionary)
* id_line (a line ID that begins with the six-digit P, Q, or X number of the text)
* id_text (six-digit P, Q, or X number)
* id_word (word ID that begins with the ID number of the line)
* label (traditional line number in the form o ii 2' (obverse column 2 line 2'), etc.)
* lang (language code, including sux, sux-x-emegir, sux-x-emesal, akk, akk-x-stdbab, etc)
* morph (Morphology; Sumerian only)
* norm (Normalization: Akkadian)
* norm0 (Normalization: Sumerian)
* pos (Part of Speech)
* sense (contextual meaning)
* signature (full ORACC signature)

Not all data elements (columns) are available for all words. Sumerian words never have a `norm`, Akkadian words do not have `norm0`, `base`, `cont`, or `morph`. Most data elements are only present when the word is lemmatized; only `lang`, `form`, `pos`, `id_word`, `id_line`, and `id_text` should always be there. An unlemmatized word has `pos` 'X' (for unknown). Broken words have `pos` 'u' (for 'unlemmatizable).

In [51]:
#words[words['label'] == '']

# Manipulate
The columns may be manipulated with standard Pandas methods to create the desired output. By way of example, the following code will create a column `lemma` with the format **cf[gw]pos** (for instance **lugal[king]N**). For words that have no lemmatization `lemma` equals `form`. Only Sumerian words are allowed (and thus `lang` can be omitted) and in addition to the column `lemma` the column `base` is preserved; words that have no lemmatization take `form` as their base. Words and bases are concatenated to lines.

## Remove  non-Sumerian words

In [52]:
lang = ['sux', ''] # note that 'lang' is empty in entries that indicate damage
words = words.loc[words['lang'].str[:3].isin(lang)].reset_index()

## Create Lemma Column and Adjust Base

In [58]:
words['lemma'] = words['cf'] # first element of lemma is the citation form
words['lemma'] = [words['lemma'][i] + '[' + words['gw'][i] 
                     + ']' + words['pos'][i] 
                     if not words['lemma'][i] == '' 
                     else words['form'][i] +'[NA]NA' for i in range(len(words))]
words['lemma'] = [lemma if not lemma == '[NA]NA' else '' for lemma in words['lemma'] ]
words['base'] = [words['base'][i] if not words['base'][i] == '' 
                 or words['label'][i] == '' else words['form'][i] 
                 for i in range(len(words))]
#lemmas = words[['lemma', 'id_text', 'id_line', 'id_word', 'label']]#, 'extent', 'scope']]
# this is crreating errors for cases where 'extent' and 'scope' are not available 
#(because of none of the texts scraped have such entries). Create dummy fields, then fill them.
lemmas = words[['lemma', 'base', 'id_text', 'id_line', 'id_word', 'label', 'extent', 'scope']]
lemmas.head()

KeyError: "['extent' 'scope'] not in index"

## Group by Line

In [59]:
lines = words.groupby([words['id_line'], words['label']]).agg({
        'lemma': ' '.join,
        'base': ' '.join,
        'extent': ''.join, 
        'scope': ''.join
    }).reset_index()
        

KeyError: 'extent'

In [60]:
df = lines[['id_line', 'label', 'lemma', 'base', 'extent', 'scope']]
df

,id_line,label,lemma,base,extent,scope
0,P224988.4,o i 1,gubi[bird]N,gu-bi{mušen},,
1,P224988.5,o i 2,u[bird]N,u₂{mušen},,
2,P224988.6,o i 3,esig[bird]N,u₂-sig{mušen},,
3,P225020.4,o i 1,hiz[vegetable]N,hi-iz{sar},,
4,P225020.5,o i 2,hizurbara[vegetable]N,hi-iz-ur-bar-ra{sar},,
5,P225020.6,o i 3,hiz[vegetable]N dug[good]V/i,hi-iz dug₃,,
6,P225024.3,o 1,dušia[stone]N,{na₄}du₈-ši-a,,
7,P225024.4,o 2,kišib[seal]N dušia[stone]N,{na₄}kišib du₈-ši-a,,
8,P225024.5,o 3,lagab[block]N dušia[stone]N,{na₄}lagab du₈-ši-a,,
9,P225027.4,o i 1,šagida[garment]N,{tug₂}šag₄-gi-dab₅,,


In [61]:
df['id_text'] = df['id_line'].str[:7]
df['line'] = [int(re.sub('.+\.', '', line)) for line in df['id_line']] #create a line number for sorting
df = df.sort_values(['id_text', 'line']).reset_index(drop=True)
df

//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
//anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


,id_line,label,lemma,base,extent,scope,id_text,line
0,P224988.4,o i 1,gubi[bird]N,gu-bi{mušen},,,P224988,4
1,P224988.5,o i 2,u[bird]N,u₂{mušen},,,P224988,5
2,P224988.6,o i 3,esig[bird]N,u₂-sig{mušen},,,P224988,6
3,P225020.4,o i 1,hiz[vegetable]N,hi-iz{sar},,,P225020,4
4,P225020.5,o i 2,hizurbara[vegetable]N,hi-iz-ur-bar-ra{sar},,,P225020,5
5,P225020.6,o i 3,hiz[vegetable]N dug[good]V/i,hi-iz dug₃,,,P225020,6
6,P225024.3,o 1,dušia[stone]N,{na₄}du₈-ši-a,,,P225024,3
7,P225024.4,o 2,kišib[seal]N dušia[stone]N,{na₄}kišib du₈-ši-a,,,P225024,4
8,P225024.5,o 3,lagab[block]N dušia[stone]N,{na₄}lagab du₈-ši-a,,,P225024,5
9,P225027.4,o i 1,šagida[garment]N,{tug₂}šag₄-gi-dab₅,,,P225027,4


## Save in CSV Format

In [62]:
filename = '../data/' + filename[:-4] + '.csv'
print('saving ' + filename)
with open(filename, 'w') as w:
    df.to_csv(w, encoding='utf8')

saving ../data/Q42_par.csv
